# 1.) Create a function that takes X, y data and allows you to fit/predict similar to a scikit learn function.

In [15]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

class SimpleRNNWrapper:
    def __init__(self, output_type='continuous', lags=1, rnn_units=50, epochs=10, batch_size=32):
        """
        Initialize the RNN wrapper.

        Parameters:
        - output_type: 'continuous' for regression tasks, 'discrete' for classification.
        - lags: Number of lag observations to use as input features.
        - rnn_units: Number of units in the RNN layer.
        - epochs: Number of epochs to train the model.
        - batch_size: Batch size for training.
        """
        self.output_type = output_type
        self.lags = lags
        self.rnn_units = rnn_units
        self.epochs = epochs
        self.batch_size = batch_size
        self.model = None
        self.scaler = MinMaxScaler()
        self.label_encoder = None
        if output_type == 'discrete':
            self.label_encoder = LabelEncoder()

    def _preprocess_data(self, X, y=None):
        """
        Preprocess the data: scale features, encode labels (if discrete), and structure into sequences.
        """
        X_scaled = self.scaler.fit_transform(X)
        X_seq = np.array([X_scaled[i:i+self.lags] for i in range(len(X_scaled)-self.lags)])

        if y is not None:
            if self.output_type == 'continuous':
                y_seq = y[self.lags:]
            else:  # Discrete
                y_encoded = self.label_encoder.fit_transform(y)
                y_seq = to_categorical(y_encoded[self.lags:])
            return X_seq, y_seq

        return X_seq

    def _build_model(self, input_shape, output_shape):
        """
        Build the RNN model based on specified parameters.
        """
        self.model = Sequential()
        self.model.add(SimpleRNN(self.rnn_units, input_shape=input_shape))
        if self.output_type == 'continuous':
            self.model.add(Dense(1))
        else:  # Discrete
            self.model.add(Dense(output_shape, activation='softmax'))

        self.model.compile(optimizer='adam', loss='mse' if self.output_type == 'continuous' else 'categorical_crossentropy', metrics=['accuracy'])

    def fit(self, X, y):
        """
        Fit the RNN model to the data.
        """
        X_seq, y_seq = self._preprocess_data(X, y)
        self._build_model(input_shape=(X_seq.shape[1], X_seq.shape[2]), output_shape=y_seq.shape[1])
        self.model.fit(X_seq, y_seq, epochs=self.epochs, batch_size=self.batch_size)

    def predict(self, X):
        """
        Predict using the trained RNN model.
        """
        X_seq = self._preprocess_data(X)
        predictions = self.model.predict(X_seq)
        if self.output_type == 'discrete':
            return self.label_encoder.inverse_transform(np.argmax(predictions, axis=1))
        return predictions.flatten()

